# Bead detection and quantification in cells on a HPC

Detect $3\mu m$ beads inside cells and measure intensity of the signal on the 
beads in all channels. The bead detection is done using normalized cross 
correlation and the cell segmentation is achieved using cellpose.



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
cluster = SLURMCluster(
                queue="cpu",
                cores=112,
                processes=1,
                memory="700GB",
                local_directory="$SLURM_SCRATCH_DIR",
                shebang="#!/usr/bin/env tcsh",               
                walltime="72:00:00",                
                death_timeout=150,
            )
cluster.adapt(maximum_jobs=10)
client = Client(cluster)

In [3]:
print(cluster.job_script())

#!/usr/bin/env tcsh

#SBATCH -J dask-worker
#SBATCH -p cpu
#SBATCH -n 1
#SBATCH --cpus-per-task=28
#SBATCH --mem=140G
#SBATCH -t 72:00:00

/lmb/home/jeromeb/miniconda3/envs/imaging/bin/python -m distributed.cli.dask_worker tcp://10.1.0.29:32949 --nthreads 28 --memory-limit 139.70GiB --name dummy-name --nanny --death-timeout 150 --local-directory $SLURM_SCRATCH_DIR



In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.1.0.29:8787/status,
Dashboard: http://10.1.0.29:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.1.0.29:32949,Workers: 0
Dashboard: http://10.1.0.29:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
import beadfinder

In [6]:
bf = beadfinder.FolderProcessor('config1.yml')

Source folder accessible? True
Destination folder accessible? True
Slide1_B3.nd2
Slide1_A3.nd2
Slide1_A2.nd2
Slide1_A4.nd2
Slide1_A1.nd2
Discovered 245 positions in 5 files.


In [7]:
#bf.filelist.to_csv(bf.destination/'filelist.csv')

In [8]:
bf.process_parallel_dataset()

KeyboardInterrupt: 

In [8]:
import pandas as pd
bf = beadfinder.FolderProcessor('config2.yml')
bf.filelist = pd.read_csv(bf.destination/'filelist.csv')

Source folder accessible? True
Destination folder accessible? True
Slide2_A3.nd2
Slide2_A1.nd2
Slide2_B1.nd2
Slide1_B3.nd2
Slide2_B3.nd2
Slide1_B2.nd2
Slide1_A3.nd2
Slide1_A2.nd2
Slide1_B1.nd2
Slide1_B4.nd2
Slide1_A4.nd2
Slide1_A1.nd2
Discovered 972 positions in 12 files.


In [9]:
bf.process_parallel_dataset()

RuntimeError: Can't redefine method: __getstate__ on class: __torch__.torch.utils.mkldnn.MkldnnBatchNorm (of Python compilation unit at: 0x55e46d521380)

In [ ]:
cluster.scale(0)
client.shutdown()